In [86]:
# reload extension
%load_ext autoreload
%autoreload 2

from processingPipe import ImageProcessingPipe
import napari
from skimage.io import imread
from magicgui import magicgui
from napari.types import ImageData, LabelsData
import pyclesperanto_prototype as cle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [100]:
image = imread("test.tif")
cle.select_device("RTX")

<NVIDIA GeForce RTX 3060 Laptop GPU on Platform: NVIDIA CUDA (1 refs)>

In [107]:
@magicgui(auto_call=True)
def process_image(img: ImageData, 
                  h: float = 1, 
                  patch_size: int = 7, 
                  window_size: int = 21, 
                  lower_thresh: int = 0, 
                  upper_thresh: int = 70) -> LabelsData:
    pipe = ImageProcessingPipe()
    pipe.create([("downscale_2x", {}),
                 # ("gaussian_blur", {"ksize": (blur_size, blur_size), "sigmaX": sigma}),
                 ("to_8bit", {}),
                 ("nlm_blur", {"h": h, "templateWindowSize": patch_size, "searchWindowSize": window_size}),
                 ("threshold_between", {"lower": lower_thresh, "upper": upper_thresh}),
                 ("upscale_2x", {}),
                 # ("dilate", {"ksize": (ksize, ksize)}),

                 ])
    pipe.run(img)
    img = pipe.results[6].image
    # img = cle.connected_components_labeling_box(img)
    return img

# process_image(image)

In [105]:
from processingPipe import *
@magicgui(auto_call=True)
def process_image(img: ImageData,
                  h: float = 1,
                  patch_size: int = 7,
                  window_size: int = 21,
                  lower_thresh: int = 0,
                  upper_thresh: int = 70) -> LabelsData:
    img = downscale_2x(img)
    img = to_8bit(img)
    img = nlm_blur(img, {"h": h, "templateWindowSize": patch_size, "searchWindowSize": window_size})
    thresholded = threshold_between(img, {"lower": lower_thresh, "upper": upper_thresh})
    img = upscale_2x(thresholded)
    # img = cle.connected_components_labeling_box(img)
    return img

In [108]:
# start up napari
# @magicgui(auto_call=True)
# def process_image(image_input: ImageData, sigma: float = 2) -> LabelsData:
#     if input is not None:
#         # process the image
#         blurred = cle.gaussian_blur(image_input, sigma_x=sigma, sigma_y=sigma)
#         binary = cle.threshold_otsu(blurred)
#         labels = cle.connected_components_labeling_box(binary)

        # return labels
viewer = napari.Viewer()
viewer.add_image(image, name='blobs')

# generate a Graphical User Interface from the function above magically
viewer.window.add_dock_widget(process_image)